In [6]:
#importing some useful packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
from scipy.stats import linregress
import glob
import pickle

#importing notebook gui library
# import ipywidgets as widgets
# from ipywidgets.widgets import Label, FloatSlider, Button, IntSlider
# from ipywidgets import Layout, HBox, VBox
# from IPython.display import display
# from ipywidgets import interact
# import time
# from matplotlib.widgets import Slider, Button

%matplotlib inline

# Advanced Lane Line

* Compute the camera calibration matrix and distortion coefficients given a set of chessboard images.

## Calibrate camera

In [67]:
nx = 9
ny = 6

images = glob.glob('./camera_cal/calibration*.jpg')
images.sort()
# print(images[3])
# images = [images[3]]

objpoints = []
imgpoints =[]

objp = np.zeros((ny*nx, 3), np.float32)
objp[:,:2] = np.mgrid[0:nx,0:ny].T.reshape(-1,2)
# print(objp)

# fig = plt.figure(figsize=(15,15))
# fig.set_tight_layout(tight=True)
# ncols = 2
# nrows = 20

# Calibrate images in camera_cal\
for i, fname in enumerate(images):
    savfname = "{}{}{}.jpg".format('./camera_cal_output/',(fname.split(sep='/')[-1]).split(sep='.')[0], "_points")
    
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    ret, corners = cv2.findChessboardCorners(gray, (nx, ny), None)

    if ret == True:
        # Found cornders
        objpoints.append(objp)
        imgpoints.append(corners)
        
        # Draw on image and save 
        cv2.drawChessboardCorners(img, (nx, ny), corners, ret)
        cv2.imwrite(savfname, img)

# Caculate calibration coefficient from all images have points.
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)
        
# Correct images found  and save as "_cal_output"
for fname in images:
    savfname = "{}{}{}.jpg".format('./camera_cal_output/',(fname.split(sep='/')[-1]).split(sep='.')[0], "_cal")
    img = cv2.imread(fname)
    undist = cv2.undistort(img, mtx, ret, None, mtx)  
    cv2.imwrite(savfname, undist)

    

### Calibration Result

* Apply a distortion correction to raw images.

#### Before :
Grid corners identified and draw to original image
![before](./camera_cal_output/calibration2_points.jpg)

#### After:
Distort image from calibrated matrices
![after](./camera_cal_output/calibration2_cal.jpg)


## Lane Detection

* Use color transforms, gradients, etc., to create a thresholded binary image.